<a href="https://colab.research.google.com/github/FarrazNouval/data-science-notebooks/blob/main/farraz_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.age.value_counts()

In [ ]:
import seaborn as sns
fig, axes = plt.subplots(3, 2, figsize=(14, 15))
fig.suptitle('Continous Data Distribution')

axes[0, 0].hist(x=dataset.age)
axes[0, 0].set_xlabel('age')
axes[0, 0].set_ylabel('frequencies')

sns.boxplot(x=dataset.age, ax=axes[0, 1])
axes[0, 1].set_xlabel('age')

axes[1, 0].hist(x=dataset.bmi)
axes[1, 0].set_xlabel('bmi')
axes[1, 0].set_ylabel('frequencies')

sns.boxplot(x=dataset.bmi, ax=axes[1, 1])
axes[1, 1].set_xlabel('bmi')

axes[2, 0].hist(x=dataset.expenses)
axes[2, 0].set_xlabel('expenses')
axes[2, 0].set_ylabel('frequencies')

sns.boxplot(x=dataset.expenses)
axes[2, 1].set_xlabel('expenses');


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('Discrete and Categorical Data Distribution')

sns.countplot(x=dataset.children, ax=axes[0, 0])
axes[0, 0].set_xlabel('number of childrens')
axes[0, 0].set_ylabel('frequencies')

sns.countplot(x=dataset.smoker, ax=axes[0, 1])
axes[0, 1].set_xlabel('is smoker ?')
axes[0, 1].set_ylabel('frequencies')

sns.countplot(x=dataset.sex, ax=axes[1, 0])
axes[1, 0].set_xlabel('sex')
axes[1, 0].set_ylabel('frequencies')


sns.countplot(x=dataset.region, ax=axes[1, 1])
axes[1, 1].set_xlabel('region')
axes[1, 1].set_ylabel('frequencies');


In [ ]:
cols = dataset.columns
input_cols = list(cols)[:len(cols)-1]
target = 'expenses'

inputs_df = dataset[input_cols].copy()
target = dataset[target]

print(type(inputs_df))
inputs_df

In [ ]:
print(type(target))
target

In [ ]:
numeric_cols = list(inputs_df.dtypes[inputs_df.dtypes != 'object'].index)
categorical_cols = list(inputs_df.dtypes[inputs_df.dtypes == 'object'].index)

num_data = inputs_df[numeric_cols]
cat_data = inputs_df[categorical_cols]


In [ ]:
num_data


In [ ]:
cat_data

In [ ]:
import sklearn 
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(cat_data)
encoded_cols = list(encoder.get_feature_names(categorical_cols))
inputs_df[encoded_cols] = encoder.transform(cat_data)


In [ ]:
inputs_df

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_dataset, train_labels, test_labels = train_test_split(inputs_df[numeric_cols + encoded_cols],
                                                                            target,
                                                                            test_size=0.2,
                                                                            random_state=42)


In [ ]:
train_features


In [ ]:
train_labels

In [ ]:
model = keras.Sequential([layers.Input(shape=(len(train_features.keys()),)),
                          layers.Normalization(axis=-1),
                          layers.Dense(11, activation='relu'),
                          layers.Dropout(0.1),
                          layers.Dense(64, activation='relu'),
                          layers.Dropout(0.1),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(1, activation='relu')])




In [ ]:
model.compile(loss='mean_absolute_error',
              optimizer=keras.optimizers.RMSprop(0.001),
              metrics=['mae', 'mse']) 

In [ ]:
#setting callbacks
#reduce learning rate
set_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                           patience=7,
                                           min_lr=0.0001,
                                           verbose=1)

#set checkpoint
set_cp = keras.callbacks.ModelCheckpoint(monitor='loss',
                                         filepath='/content/model.h5',
                                         save_best_only=True,
                                         verbose=1)
#custom callback for early stopping
class SpecificStopper(keras.callbacks.Callback):
  def __init__(self, exp_loss, exp_val_loss):
    super(SpecificStopper).__init__()
    self.exp_loss = exp_loss
    self.exp_val_loss = exp_val_loss
  def on_epoch_end(self, epoch, logs={}):
    loss = logs.get('loss')
    val_loss = logs.get('val_loss')
    if ((loss <= self.exp_loss) and (val_loss <= self.exp_val_loss)):
      print('early stopping')
      self.model.stop_training = True

set_stop = SpecificStopper(3500, 3500)

In [ ]:
history = model.fit(train_features,
                    train_labels,
                    validation_split=0.2,
                    verbose=1,
                    epochs=1000,
                    callbacks=[set_lr, set_cp, set_stop])

In [ ]:
model = keras.models.load_model('/content/model.h5')


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
